# Clustering of OULAD dataset

In [50]:
import os
import time
import pandas as pd
import numpy as np
import missingno as msno
from sklearn.impute import SimpleImputer
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pylab as pl
import subprocess
import itertools

from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import Birch
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import jaccard_score
from sklearn_extra.cluster import KMedoids
from IPython.display import clear_output

sns.set()
pd.options.display.max_columns = None

## Charging the dataset into memory

In [2]:
assessments_df         = pd.read_csv('./OULAD/assessments.csv')
courses_df             = pd.read_csv('./OULAD/courses.csv')
studentAssessment_df   = pd.read_csv('./OULAD/studentAssessment.csv')
studentInfo_df         = pd.read_csv('./OULAD/studentInfo.csv')
studentRegistration_df = pd.read_csv('./OULAD/studentRegistration.csv')
studentVle_df          = pd.read_csv('./OULAD/studentVle.csv')
vle_df                 = pd.read_csv('./OULAD/vle.csv')

# we store references
dataset_dict = {
    'assessments': assessments_df,
    'courses': courses_df,
    'studentAssessment': studentAssessment_df,
    'studentInfo': studentInfo_df,
    'studentRegistration': studentRegistration_df,
    'studentVle': studentVle_df,
    'vle': vle_df
}

## Filling NaNs / Filtering / Restructuring

In [3]:
# some of imd_band are NaN -> replacing them with 0-100%
imp = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value='0-100%')
studentInfo_df['imd_band'] = imp.fit_transform(studentInfo_df[['imd_band']])

# 82% of vle.week_from vle.week_to are NaN
# of the remaining 18% rows - in 99.2% of cases vle.week_from is equal to vle.week_to
# with this in mind - we can ignore this data
del vle_df['week_from']
del vle_df['week_to']

### Filtering out one specific course and merging all tables into one

In [4]:
def getOneCourse(dataset_dict, code_module, code_presentation):
    '''
    Merge all tables by their primary keys for a single course,
    replace NaNs of missing Exam/unregistration dates with module_presentation_length
    '''
    def filterByCode(table):
        '''returns a boolean pd.Series of same table length'''
        return  (table['code_module'] == code_module) & \
                (table['code_presentation'] == code_presentation)
    
    course = dataset_dict['courses']
    course = course[filterByCode(course)]
    module_presentation_length = course['module_presentation_length'].values[0]
    
    # studentAssessment -> complete studentAssessments with assessments info
    assessments = dataset_dict['assessments']
    assessments.loc[ (assessments['assessment_type'] == 'Exam') & \
        filterByCode(assessments), \
        'date'] = module_presentation_length
    assessments = assessments[filterByCode(assessments)]
    studentAssessment = pd.merge(dataset_dict['studentAssessment'], assessments, \
                                 how='inner', on='id_assessment')

    # studentInfo -> complete studentInfo with studentRegistration
    studentRegistration = dataset_dict['studentRegistration']
    studentRegistration.loc[studentRegistration['date_unregistration'].isna() & \
        filterByCode(studentRegistration), \
        'date_unregistration'] = module_presentation_length
    studentRegistration = studentRegistration[ \
                           filterByCode(studentRegistration)]
    studentInfo = pd.merge(dataset_dict['studentInfo'], studentRegistration, \
                           how='inner', on=['id_student', 'code_module', 'code_presentation'])

    # studentVle = complete vle with studentVle
    vle = dataset_dict['vle']
    vle = vle[filterByCode(vle)]
    studentVle = pd.merge(dataset_dict['studentVle'], vle, \
                          how='inner', on=['id_site', 'code_module', 'code_presentation'])
    del studentVle['id_site']
    
    # complete with studentVle with studentInfo, complete studentAssessment with studentInfo
    # then append enhanced studentVle with studentAssessment, sort by date
    studentVleInfo = pd.merge(studentVle, studentInfo, \
                              how='inner', on=['id_student', 'code_module', 'code_presentation'])
    studentAssessmentInfo = pd.merge(studentAssessment, studentInfo, \
                              how='inner', on=['id_student', 'code_presentation', 'code_module'])
    
    combined_df = studentAssessmentInfo.append(studentVleInfo)
    del combined_df['code_module']
    del combined_df['code_presentation']
    del combined_df['id_assessment']
    combined_df = combined_df.sort_values(by=['date'])
    return combined_df

### Restructuring the oneCourse table
- oneCourse table might be good for algorithms that take into account the sequence / time
- to make a first POC we will simplify drasticaly
    - remove every thing that is beyond day 14
    - aggregate each student sequence to make for each student contain only one row
        - date_submitted (ignore NaNs) -> compute the mean 
        - same with date, score
        - is_banked (ignore NaNs) -> compute the sum (= how many assessments were banked)
        - same with score
        - assessment_type -> create 2 variables: 
            - sumTMA -> sum of TMAs
            - sumCMA -> sum of CMAs
        - for each activity type make a variable and take the sum of sum_click

In [5]:
def restructure(oneCourse):
    '''
    aggregate each student sequence to make for each student contain only one row,
    keeping only the data of the first two weeks
    '''
    # prediction is only then interresting when it's the begining of the course - not the end!
    first14Days_oneCourse = oneCourse[oneCourse['date'] <= 14]
    # remove those who unregistered before the begining as we can't do anything for them
    first14Days_oneCourse = first14Days_oneCourse[first14Days_oneCourse['date_unregistration'] \
                                                  > 0]
    # list of unique activity_types to create features
    activity_types_df = first14Days_oneCourse['activity_type'].unique()
    # remove NaN activity type
    activity_types_df = [x for x in activity_types_df if type(x) == str]
    # we want one student per line (the easy way)
    final_df = first14Days_oneCourse.groupby('id_student').agg({
        'score': [np.mean, np.sum],
        'date_submitted': [np.mean],
        'is_banked': [np.sum],
        'assessment_type': [('CMA_count', lambda x: x.values[x.values == 'CMA'].size), \
                            ('TMA_count', lambda x: x.values[x.values == 'TMA'].size)],
        'date':[np.mean],
        'weight': [np.mean, np.sum],
        'gender': [('first', lambda x: x.values[0])],
        'region': [('first', lambda x: x.values[0])],
        'highest_education': [('first', lambda x: x.values[0])],
        'imd_band': [('first', lambda x: x.values[0])],
        'age_band': [('first', lambda x: x.values[0])],
        'num_of_prev_attempts': [('first', lambda x: x.values[0])],
        'studied_credits': [('first', lambda x: x.values[0])],
        'disability': [('first', lambda x: x.values[0])],
        'final_result': [('first', lambda x: x.values[0])],
        'date_registration': [('first', lambda x: x.values[0])],
        'date_unregistration': [('first', lambda x: x.values[0])],
        'sum_click': [np.mean, np.sum],
        'activity_type': [('list', lambda x: [x.values[x.values == activity].size \
                                              for activity in activity_types_df])]
    })
    # keeping only one level of columns names
    final_df.columns = ["_".join(x) for x in final_df.columns.ravel()]
    custom_columns = ['activity_type_' + str(x) for x in activity_types_df]
    # splitting & concatenating the created features (there should be a better way to do it...)
    final_df = final_df.join(pd.DataFrame(final_df.activity_type_list.values.tolist(), \
                                          columns=custom_columns, index=final_df.index))
    del final_df['activity_type_list']
    return final_df

### Replace NaNs introduced by new features
#### and map categorical_columns to numbers

In [6]:
def cleanAndMap(final_df):
    '''
    replace NaNs intoduced by new features and map catogorical columns to numbers
    returning the encoder object to decode the labels later on
    '''
    # replacing NaNs
    final_df.loc[final_df['weight_mean'].isna(), 'weight_mean'] = 0
    final_df.loc[final_df['score_mean'].isna(), 'score_mean'] = 0
    final_df.loc[final_df['date_submitted_mean'].isna(), 'date_submitted_mean'] = -1

    # replacing categorical_columns with numbers
    categorical_columns = ['gender_first', 'highest_education_first', 'imd_band_first', \
                           'age_band_first', 'disability_first', 'final_result_first',  \
                           'region_first']

    encoders = {col: LabelEncoder() for col in categorical_columns }
    for col in categorical_columns:
        final_df.loc[:, col] = encoders[col].fit_transform(final_df[col])
    
    return encoders

In [7]:
code_module = 'CCC'
code_presentation = '2014B'
oneCourse = getOneCourse(dataset_dict, code_module, code_presentation)
final_df = restructure(oneCourse)
encoders = cleanAndMap(final_df)

# some information about the final_df
print(final_df.describe(include='all'))
print('REGION REPARTITION')
print(final_df.groupby('region_first').size())
print('EDUCATION REPARTITION')
print(final_df.groupby('highest_education_first').size())
print('FINAL RESULT REPARTITION')
print(final_df['final_result_first'].value_counts())
# to check that we don't have NaNs!
print('final_df contains %i NaNs' % sum(final_df.isna().sum()))

       score_mean  score_sum  date_submitted_mean  is_banked_sum  \
count      1577.0     1577.0               1577.0         1577.0   
mean          0.0        0.0                 -1.0            0.0   
std           0.0        0.0                  0.0            0.0   
min           0.0        0.0                 -1.0            0.0   
25%           0.0        0.0                 -1.0            0.0   
50%           0.0        0.0                 -1.0            0.0   
75%           0.0        0.0                 -1.0            0.0   
max           0.0        0.0                 -1.0            0.0   

       assessment_type_CMA_count  assessment_type_TMA_count    date_mean  \
count                     1577.0                     1577.0  1577.000000   
mean                         0.0                        0.0     1.723750   
std                          0.0                        0.0     5.068600   
min                          0.0                        0.0   -18.000000   
25%    

## Clustering

#### IDEA:
- imagine the course just started - we are at week 2
- we want to identify students which might Fail/Withdraw to propose them help
- to start of simple we don't try to predict WHEN a student will Fail/Withdraw and we ignore all student interactions with other courses
- we suppose that students that Fail/Withdraw can be distinguished by their behaviour and features but we don't know which ones in advance
- with clustering we can identify groups of student that are similar
- we suppose that students that have similar behaviour / features would have similar outcomes

- we want to measure if the resulting groups could provide useful information for a set of prediction algorithms thereby improving their predictions 

- how does the quality of the generated groups impact the predictors?

- could a consensus clustering algorithm produce an even better set of groups to improve even more the prediction?
- how does the clustering algorithms generalise for differenes courses / the same course the next year?
- if each time the optimal hyper params are different - could the consensus clustering 
   algorithm solve this issue by combining multiple choices of hyper params at the same time?

#### Separate response label from training set

In [8]:
# another solution would be to cluster a previous course (training) and use the current one for
#   prediction ... for now we want just to measure the clustering quality on all the data
# train, test = train_test_split(final_df, random_state=0)
scaler = MinMaxScaler()
trainX = final_df.drop(['final_result_first'], axis=1)
trainX = scaler.fit_transform(trainX)
testY = final_df['final_result_first']

#### Custom clustor quality measure

In [9]:
def customClusteringScore(encoders, clustAlgo_labels, testY):
    new_col_list = encoders['final_result_first'].inverse_transform([0,1,2,3])
    compare_df = pd.concat([testY, clustAlgo_labels], axis=1)
    # bring back the final_result labels (not really needed but makes the code clearer)
    compare_df.loc[:,'final_result_first'] = \
        encoders['final_result_first'].inverse_transform(compare_df.loc[:,'final_result_first'])
    compare_df = compare_df.groupby([clustAlgo_labels.name, 'final_result_first']).size()
    # the adjusted_rand_score measure is good for measuring the quality of the cluster
    #   in general
    # print('adjusted_rand_score: ', adjusted_rand_score(testY, kmeans_labels))

    # but we want to measure how good the clustering algorithm is able to separate 
    #  Distinction/Pass from Fail/Withdrawn:
    # the measure should be close or equal to 0 if both groups are almost equal or equal in size
    # should be close or equal to 1 if one of the groups is small compared to the other
    # should be between (0,1) if g1 > g2 / g1 < g2
    # proposed measure: score = abs(g1 - g2) / g1 + g2
    # we want to take into account the size of the groups too
    # we prefer 'big' groups with a good score rather than small groups with exellent scores
    # for that we could normalize the group score : score * group-size / total size
    # a perfect clustering would be one that has only 2 groups each of them of score 1
    # small groups would have a little weight compared to others but can sum up to 1
    # for now let try not to penalize by group count...
    nb_clusters = clustAlgo_labels.nunique()
    scores = []
    for cluster in range(0,nb_clusters):
        failWihdrawn = compare_df[cluster]['Fail'] \
                            if 'Fail' in compare_df[cluster].index else 0
        failWihdrawn += compare_df[cluster]['Withdrawn'] \
                            if 'Withdrawn' in compare_df[cluster].index else 0
        PassDistinction = compare_df[cluster]['Pass'] \
                            if 'Pass' in compare_df[cluster].index else 0
        PassDistinction += compare_df[cluster]['Distinction'] \
                            if 'Distinction' in compare_df[cluster].index else 0
        scores.append(\
            (abs(PassDistinction - failWihdrawn) / (PassDistinction + failWihdrawn)) *\
                      (PassDistinction + failWihdrawn) / len(testY))
    return sum(scores)

#### Used clustering algorithms and their hyperparameters

In [10]:
kMedoidsParams = {
    'vanilla' : {},
    # removing to improve performances
#     'init=k-medoids++': {'init': 'k-medoids++'},
#     'metric=l1': {'metric': 'l1'},
#     'metric=l2': {'metric': 'l2'},
#     'metric=manhattan': {'metric': 'manhattan'}
}

kmeansParams = {
    'vanilla' : {},
    'tol=5': {'tol': 1e-5},
     # removing to improve performance
#     'n_init30' : {'n_init': 30},
#     'max_iter=400': {'max_iter': 400},
#     'max_iter=600': {'max_iter': 600},
#     'max_iter=400 tol=5': {'max_iter': 400, 'tol': 1e-5},
#     'max_iter=600 tol=5': {'max_iter': 600, 'tol': 1e-5},
#     'max_iter=400 tol=6': {'max_iter': 400, 'tol': 1e-6},
#     'max_iter=600 tol=6': {'max_iter': 600, 'tol': 1e-6},
#     'n_init15 max_iter=400' : {'n_init': 15, 'max_iter': 400},
#     'n_init15 max_iter=600' : {'n_init': 15, 'max_iter': 600},
    
    # removing because of low/similar results
#     'tol=6': {'tol': 1e-6},
#     'n_init15' : {'n_init': 15},
#     'n_init60' : {'n_init': 60},
#     'max_iter=800': {'max_iter': 800},
#     'max_iter=800 tol=5': {'max_iter': 800, 'tol': 1e-5},
#     'max_iter=800 tol=6': {'max_iter': 800, 'tol': 1e-6},
#     'n_init15 max_iter=800' : {'n_init': 15, 'max_iter': 800},
#     'n_init30 max_iter=400' : {'n_init': 30, 'max_iter': 400},
#     'n_init30 max_iter=600' : {'n_init': 30, 'max_iter': 600},
#     'n_init30 max_iter=800' : {'n_init': 30, 'max_iter': 800},
#     'n_init60 max_iter=400' : {'n_init': 60, 'max_iter': 400},
#     'n_init60 max_iter=600' : {'n_init': 60, 'max_iter': 600},
#     'n_init60 max_iter=800' : {'n_init': 60, 'max_iter': 800},
}

spectralClusteringParams = {
    'affinity=laplacian': {'affinity': 'laplacian'},
    # removing to improve performance
#     'vanilla' : {},
#     'eigen_solver=arpack': {'eigen_solver': 'arpack'},
#     'eigen_solver=amg': {'eigen_solver': 'amg'},
#     'assign_labels=discretize': {'assign_labels': 'discretize'},
#     'eigen_solver=lobpcg': {'eigen_solver': 'lobpcg'}, # not working for k>=6
}

agglomerativeClusteringParams = {
    'vanilla' : {},
    
    # removing to improve performance
    'linkage=average affinity=l1' : {'linkage': 'average', 'affinity': 'l1'},
    'linkage=average affinity=manhattan' : {'linkage': 'average', 'affinity': 'manhattan'},
    
    # removing because of low results
    'linkage=complete' : {'linkage': 'complete'},
    'linkage=average' : {'linkage': 'average'},
    'linkage=single' : {'linkage': 'single'},
    'linkage=complete affinity=l1' : {'linkage': 'complete', 'affinity': 'l1'},
    'linkage=single affinity=l1' : {'linkage': 'single', 'affinity': 'l1'},
    'linkage=complete affinity=l2' : {'linkage': 'complete', 'affinity': 'l2'},
    'linkage=average affinity=l2' : {'linkage': 'average', 'affinity': 'l2'},
    'linkage=single affinity=l2' : {'linkage': 'single', 'affinity': 'l2'},
    'linkage=complete affinity=manhattan' : {'linkage': 'complete', 'affinity': 'manhattan'},
    'linkage=single affinity=manhattan' : {'linkage': 'single', 'affinity': 'manhattan'},
    'linkage=complete affinity=cosine' : {'linkage': 'complete', 'affinity': 'cosine'},
    'linkage=average affinity=cosine' : {'linkage': 'average', 'affinity': 'cosine'},
    'linkage=single affinity=cosine' : {'linkage': 'single', 'affinity': 'cosine'},
}

birchParams = {
    'vanilla': {},
}

dbscanParams = {
    'vanilla': {},
}

clustAlgos = {
    "kMeans": {
        'obj': KMeans, 
        'params': kmeansParams,
        'paramRanges': {'n_clusters' : range(3, 4)}
    },
    "kMedoits": {
        'obj': KMedoids, 
        'params': kMedoidsParams,
        'paramRanges': {'n_clusters' : range(2, 4)}
    },
    "SpectralClustering": {
        'obj': SpectralClustering,
        'params': spectralClusteringParams,
        'paramRanges': {'n_clusters' : range(3, 5)}
    },
#     "AgglomerativeClustering": {
#         'obj': AgglomerativeClustering,
#         'params': agglomerativeClusteringParams,
#         'paramRanges': {'n_clusters' : range(2, 4)}
#     },
#     "Birch": {
#         'obj': Birch, 
#         'params': birchParams,
#         'paramRanges': {'n_clusters' : range(2, 4)}
#     },
    
#     "DBSCAN": {'obj': DBSCAN, 'params': dbscanParams}, # density based - don't use nb_clusters
}

#### Computing the baseClusterings

In [11]:
# 1 Generate clustering ensemble of the dataset and store the clustering vectors in a 
#   list BaseClusterings
baseClusterings = pd.DataFrame(index=testY.index)
results = pd.DataFrame(columns=['k', 'algo', 'type', 'score'])
stepCount = 0
totalStepsCount = sum([len(clustAlgos[key]['params']) * \
                       sum([len(clustAlgos[key]['paramRanges'][innerkey]) \
                            for innerkey in clustAlgos[key]['paramRanges']]) \
                            for key in clustAlgos])

for clustAlgoKey in clustAlgos:
    for paramsKey in clustAlgos[clustAlgoKey]['params']:
        for rangedParamKey in clustAlgos[clustAlgoKey]['paramRanges']:
            for rangedParamValue in clustAlgos[clustAlgoKey]['paramRanges'][rangedParamKey]:
                # for for for for for for for for ...
                clear_output(wait=True)
                clustAlgos[clustAlgoKey]['params'][paramsKey][rangedParamKey] = \
                                rangedParamValue
                clustAlgo = clustAlgos[clustAlgoKey]['obj'](\
                                **clustAlgos[clustAlgoKey]['params'][paramsKey])
                clustAlgo.fit(trainX)
                # we don't want to predict - for now only evaluate the quality of the clustering
                # clustAlgo_predict = pd.Series(clustAlgo.predict(trainX), name=clustAlgoKey, \
                # index=testY.index)
                clustAlgo_labels = pd.Series(clustAlgo.labels_, name=clustAlgoKey, \
                                             index=testY.index)
                baseClusterings.insert(len(baseClusterings.columns), str(stepCount+1), \
                                       clustAlgo_labels)

                results = results.append({
                    'k': rangedParamValue, 
                    'algo': clustAlgoKey, 
                    'type': paramsKey, 
                    'score': customClusteringScore(encoders, clustAlgo_labels, testY) 
                    }, ignore_index=True)
                stepCount += 1
                print('step %i/%i (%i%s)' % (stepCount, totalStepsCount, \
                                             round(100 * stepCount/totalStepsCount,2), '%'))

nb_lines = 10
resultTable = pd.DataFrame(index=range(0,nb_lines))
for group in results.sort_values(by='score', ascending=False).groupby(['algo']):
    if len(group[1]) < nb_lines:
        group = (group[0], group[1].append([x for x in [0] * nb_lines]))
    
    resultTable.insert(len(resultTable.columns), "%s_k" % \
                       (group[0]), group[1].head(nb_lines)['k'].values)
    resultTable.insert(len(resultTable.columns), "%s_type" % \
                       (group[0]), group[1].head(nb_lines)['type'].values)
    resultTable.insert(len(resultTable.columns), "%s_score" % \
                       (group[0]), group[1].head(nb_lines)['score'].values)

print(baseClusterings)
resultTable

step 6/6 (100%)
            1  2  3  4  5  6
id_student                  
28418       2  0  0  0  1  3
29764       0  1  1  1  2  1
29820       0  1  1  1  2  1
40333       1  2  1  2  0  2
40604       0  1  1  1  2  1
...        .. .. .. .. .. ..
2681198     1  2  1  2  0  2
2686578     0  1  0  1  0  1
2692327     0  1  1  1  2  1
2697181     2  0  0  0  1  3
2698535     0  1  0  0  2  1

[1577 rows x 6 columns]


,SpectralClustering_k,SpectralClustering_type,SpectralClustering_score,kMeans_k,kMeans_type,kMeans_score,kMedoits_k,kMedoits_type,kMedoits_score
0,3,affinity=laplacian,0.398859,3,tol=5,0.305010,3,vanilla,0.360812
1,4,affinity=laplacian,0.302473,3,vanilla,0.303741,2,vanilla,0.242866
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### MultiCons

In [12]:
# D= {1,2,3,4,5,6,7,8,9} partitioned using five base clusterings into the five partitions:
# P1 = {{1,2,3},{4,5,6,7,8,9}},
# P2 = {{1,2,3},{4,5,6,7,8,9}},
# P3 = {{1,2,3,4,5},{6,7},{8,9}},
# P4 = {{4,5,6,7}, {1,2,3},{8,9}}
# P5 = {{4,5,6,7},{1,2,3},{8,9}}
clust = pd.DataFrame(index=range(1,10))
membershipMatrix = pd.DataFrame(index=range(1,10))
clust.insert(len(clust.columns), '1', [0,0,0,1,1,1,1,1,1])
clust.insert(len(clust.columns), '2', [0,0,0,1,1,1,1,1,1])
clust.insert(len(clust.columns), '3', [0,0,0,0,0,1,1,2,2])
clust.insert(len(clust.columns), '4', [1,1,1,0,0,0,0,2,2])
clust.insert(len(clust.columns), '5', [1,1,1,0,0,0,0,2,2])


In [13]:
baseClusterings = clust

#### Building the Membership matrix

In [14]:
# there should be a much better way to construct the membership matrix...
# for now let's use a naive implementation

# 3 Build the cluster membership matrix M
def buildMembershipMatrix(baseClusterings):
    ''' Computes and returns the Membership matrix'''
    # colDict = {}
    membershipMatrix = pd.DataFrame(index=baseClusterings.index)
    for col in baseClusterings.columns:
    #     colDict[col] = []
        for partition in np.sort(baseClusterings[col].unique()):
            membershipMatrix.insert(len(membershipMatrix.columns), '%sP%i' % (col, partition), baseClusterings[col] == partition)
    #         colDict[col].append(partition)
    #     colDict[col].append(-1)
    # colDict[col].pop()
    return membershipMatrix

membershipMatrix = buildMembershipMatrix(baseClusterings)
membershipMatrix.astype(int)

,1P0,1P1,2P0,2P1,3P0,3P1,3P2,4P0,4P1,4P2,5P0,5P1,5P2
1,1,0,1,0,1,0,0,0,1,0,0,1,0
2,1,0,1,0,1,0,0,0,1,0,0,1,0
3,1,0,1,0,1,0,0,0,1,0,0,1,0
4,0,1,0,1,1,0,0,1,0,0,1,0,0
5,0,1,0,1,1,0,0,1,0,0,1,0,0
6,0,1,0,1,0,1,0,1,0,0,1,0,0
7,0,1,0,1,0,1,0,1,0,0,1,0,0
8,0,1,0,1,0,0,1,0,0,1,0,0,1
9,0,1,0,1,0,0,1,0,0,1,0,0,1


#### Generate FCPs

In [ ]:
# me trying to mine frequent closed patterns... and resulting to mine all patterns... 
# result is very slow compared to existing FCI mining techniques
# We will not use this function!
def slowFCImining(): 
    def getPartitionFromArray(clust_array):
        res = []
        for i, val in enumerate(clust_array):
            if colDict[str(i+1)][val] >= 0:
                res.append(''.join([str(i+1), 'P', str(colDict[str(i+1)][val])]))

        return res

    def getKeyFromArray(clust_array):
        return '|'.join(getPartitionFromArray(clust_array))

    def checkAndInsertFrequentPattern(frequentPatterns, patternKeys, patternArray):
        strhash = hash(str(patternArray))
        if strhash in frequentPatterns:
            if set(frequentPatterns[strhash][0]).issubset(patternKeys):
                frequentPatterns[strhash] = [patternKeys, patternArray]

        else:
            frequentPatterns[strhash] = [patternKeys, patternArray]

    iclust = 0
    jpart = 0
    temp_clust_array = []
    memoryDict = {}
    frequentPattersDict = {}
    emptyJoin = False
    start = time.time()

    while True:
        if str(iclust + 1) in colDict and not emptyJoin:
            iclust += 1
            jpart = 0
            temp_clust_array.append(jpart)

            if len(getPartitionFromArray(temp_clust_array)) == 1:
                binaryJoin = membershipMatrix[getPartitionFromArray(temp_clust_array)[0]]
                memoryDict[getKeyFromArray(temp_clust_array)] = binaryJoin
                checkAndInsertFrequentPattern(frequentPattersDict, getPartitionFromArray(temp_clust_array),binaryJoin[binaryJoin].index.tolist())
            elif len(getPartitionFromArray(temp_clust_array)) > 1:
                binaryJoin = memoryDict[getKeyFromArray(temp_clust_array[:-1])] & membershipMatrix[getPartitionFromArray(temp_clust_array)[-1] ]
                if sum(binaryJoin) != 0:
                    checkAndInsertFrequentPattern(frequentPattersDict, getPartitionFromArray(temp_clust_array),binaryJoin[binaryJoin].index.tolist())
                    if str(iclust + 1) in colDict:
                        memoryDict[getKeyFromArray(temp_clust_array)] = binaryJoin
                else:
                    if str(iclust + 1) in colDict:
                        emptyJoin = True

        elif jpart < len(colDict[str(iclust)]) - 1 and not emptyJoin:
            jpart += 1
            temp_clust_array[len(temp_clust_array)-1] = jpart

            if getKeyFromArray(temp_clust_array[:-1]) in memoryDict:
                binaryJoin = memoryDict[getKeyFromArray(temp_clust_array[:-1])] & membershipMatrix[getPartitionFromArray(temp_clust_array)[-1] ]
                if sum(binaryJoin) != 0:
                    checkAndInsertFrequentPattern(frequentPattersDict, getPartitionFromArray(temp_clust_array),binaryJoin[binaryJoin].index.tolist())
            else:
                nojoin = membershipMatrix[getPartitionFromArray(temp_clust_array)[-1]]
                checkAndInsertFrequentPattern(frequentPattersDict, getPartitionFromArray(temp_clust_array),binaryJoin[binaryJoin].index.tolist())

        else:
            lastKeyValue = {}
            emptyJoin = False
            while len(temp_clust_array) != 0 and \
                temp_clust_array[len(temp_clust_array) - 1] == len(colDict[str(iclust)]) - 1:
                temp_clust_array.pop()
                iclust -= 1

                keyToDelete = getKeyFromArray(temp_clust_array)
                if keyToDelete in memoryDict:
                    lastKeyValue[keyToDelete] = memoryDict[keyToDelete]
                    del memoryDict[keyToDelete]

            if len(temp_clust_array) == 0:
                print('END')
                break;

            temp_clust_array[len(temp_clust_array) - 1] += 1
            if not temp_clust_array[len(temp_clust_array) - 1] == len(colDict[str(iclust)]) - 1:

                ### dublicate as above
                if len(getPartitionFromArray(temp_clust_array)) == 1:
                    memoryDict = {}
                    binaryJoin = membershipMatrix[getPartitionFromArray(temp_clust_array)[0]]
                    memoryDict[getKeyFromArray(temp_clust_array)] = binaryJoin
                    checkAndInsertFrequentPattern(frequentPattersDict, getPartitionFromArray(temp_clust_array),binaryJoin[binaryJoin].index.tolist())
                elif len(getPartitionFromArray(temp_clust_array)) > 1:
                    binaryJoin = memoryDict[getKeyFromArray(temp_clust_array[:-1])] & membershipMatrix[getPartitionFromArray(temp_clust_array)[-1] ]
                    if sum(binaryJoin) != 0:
                        checkAndInsertFrequentPattern(frequentPattersDict, getPartitionFromArray(temp_clust_array),binaryJoin[binaryJoin].index.tolist())
                    if str(iclust + 1) in colDict:
                        memoryDict[getKeyFromArray(temp_clust_array)] = binaryJoin


    end = time.time()
    print('Python elapsed time: ' + str(round((end - start), 3) * 1000) + ' ms')
    for key in frequentPattersDict:
        print(frequentPattersDict[key][0], len(frequentPattersDict[key][1]))
        
# slowFCImining()

In [23]:
# The LCM algorithm is much faster!
# thanks to https://github.com/slide-lig/plcmpp for the implementation of the LCM algorithm
# It's cloned and build in the ./FCI directory

start = time.time()
groups = list(range(0,len(membershipMatrix.columns)))
transactionList = [(membershipMatrix.iloc[x,:] * groups)[membershipMatrix.iloc[x,:]] for x in range(0, len(membershipMatrix))]

file = open('./FCI/input.txt','w')
for line in transactionList:
    file.write(str(list(line)).replace('[', '').replace(',', '').replace(']', '') + '\n')

file.close()

os.chdir(os.getcwd() + '/FCI')
# 4 Generate FCPs from M for minsupport = 0
subprocess.call("./runLCM.sh")
os.chdir(os.getcwd()[:-3])

file = open('./FCI/output.txt','r')
FCPs = []
for line in file:
    line = line.replace('\n', '')
    freq = line[:line.find('	')]
    line = np.array(list(map(int, line[line.find('	')+1:].split(' '))))
    line.sort()
    FCPs.append(list(membershipMatrix.columns[line]))
    
file.close()
end = time.time()
print('Python elapsed time: ' + str(round((end - start), 3) * 1000) + ' ms')
# 5 Sort the FCPs in ascending order according to the size of the instance sets
FCPs.sort(key = len, reverse = True)
FCPs

Python elapsed time: 18.0 ms


[['1P1', '2P1', '3P2', '4P2', '5P2'],
 ['1P0', '2P0', '3P0', '4P1', '5P1'],
 ['1P1', '2P1', '3P1', '4P0', '5P0'],
 ['1P1', '2P1', '3P0', '4P0', '5P0'],
 ['1P1', '2P1', '4P0', '5P0'],
 ['1P1', '2P1'],
 ['3P0']]

#### Define helper functions

In [71]:
def consensusFunction10(biClust):
    '''made from the Algorithm 10'''
    hasChanged = True
    while hasChanged:
        # When hasChanged is False after the iteration
        # All sets in biClust are unique
        hasChanged = False
        i = 0
        N = len(biClust)
        # using while loop because for i in range(1, N) would loop over a copy of 
        # 1,2,...,N => 1, 2,...N' and would not change if we change N
        while i < N:
            bi = biClust[i]
            # as the intersection is a symetric operation we could omit half of
            # the comparaisons? using j = i + 1 insead of j = 0
            j = i + 1
            while j < N:
                # ommiting this part as with j = i + 1 we don't enter this if statement
#                 if i == j:
#                     j += 1
#                     continue
                bj = biClust[j]
                intrscSz = bi & bj
                if len(intrscSz) == 0:
                    j += 1
                elif len(intrscSz) == len(bi):
                    # Bi⊂Bj
                    hasChanged = True
                    del biClust[i]
                    N -= 1
                elif len(intrscSz) == len(bj):
                    # Bj⊂Bi
                    hasChanged = True
                    del biClust[j]
                    N -= 1
                else:
                    biClust[j] = bi | bj
                    hasChanged = True
                    del biClust[i]
                    N -= 1

            i += 1

def assignLabel(biClust, maxDT, consVctrs):
    '''
    8 - Assign a label to each set in BiClust to build the first consensus vector and 
    store it in a list of vectors ConsVctrs
    * for convenience, ConsVctrs is a list of dictionaries that will be transformed later on 
    * in their corresponding consensus vectors
    * storing {'maxDT=5|0': {1, 2, 3}, 'maxDT=5|1': {4, 5} ...
    * instead of [0, 0, 0, 1, 1,...]
    '''
    temp = {}
    for i, partition in enumerate(biClust):
        temp['maxDT=%i|%i' % (maxDT,i)] = partition
    consVctrs.append(temp)

def jaccard(x, y):
    '''
    x and y are two dictionaries containing sets
    returns the jaccard_score (|x∩y|/|x∪y|)
    '''
    xSet = [frozenset(x[key]) for key in x]        
    ySet = [frozenset(y[key]) for key in y]
    smallerOrEqualSet, biggerOrEqualSet = (xSet, ySet) if len(xSet) < len(ySet) else (ySet, xSet)
    unionSet = set()
    intersectionSet = set()
    for i in range(0, len(biggerOrEqualSet)):
        unionSet.add(biggerOrEqualSet[i])
        if i < len(smallerOrEqualSet):
            unionSet.add(biggerOrEqualSet[i] | smallerOrEqualSet[i])
            intersectionSet.add(biggerOrEqualSet[i] & smallerOrEqualSet[i])
    return len(intersectionSet) / len(unionSet)

In [101]:
# 6 MaxDT←length(BaseClusterings)
maxDT = len(baseClusterings.columns)
# 7 BiClust ← {instance sets of FCPs built from MaxDT base clusters}
# here BiClust is a list of sets of FCPs build from maxDT baseClusters
biClust = []
filteredFCP = list(filter(lambda x: len(x) == maxDT, FCPs))
filteredFCP.sort(key = str)
consVctrs = []
for tempSet in filteredFCP:
    isInSet = True
    for col in tempSet:
        isInSet = isInSet & membershipMatrix[col]
        
    biClust.append(set(baseClusterings.index[isInSet].tolist()))
# 8 Assign a label to each set in BiClust to build the first consensus vector and store 
# it in a list of vectors ConsVctrs
assignLabel(biClust, maxDT, consVctrs)
# 9 Build the remaining consensuses
# 10 for DT = (MaxDT−1) to 1 do
for dt in range(maxDT - 1, 0, -1): 
    filteredFCP = list(filter(lambda x: len(x) == dt, FCPs))
    filteredFCP.sort(key = str)
    for tempSet in filteredFCP:
        isInSet = True
        for col in tempSet:
            isInSet = isInSet & membershipMatrix[col]
        # 11 BiClust ← BiClust ∪ {instance sets of FCPs built from DT base clusters}
        biClust.append(set(baseClusterings.index[isInSet].tolist()))
    # 12 Call the consensus function (Algo. 10)
    consensusFunction10(biClust)
    # 13 Assign a label to each set in BiClust to build a consensus vector and add it to 
    #  ConsVctrs
    assignLabel(biClust, dt, consVctrs)

# 15 Remove similar consensuses
# 16 ST ← Vector of ‘1’s of length MaxDT
st = [1] * maxDT
# 17 for i = MaxDT to 2 do
# in python the index starts with 0 -> using maxDT - 1 to 1
i = maxDT - 1
while i > 0:
    # 18 Vi ← ith consensus in ConsVctrs
    vi = consVctrs[i]
    # 19 for j = (i−1) to 1 do
    # in python the index starts with 0 -> (i−1) to 0
    j = i - 1
    while j >= 0:
        # 20 Vj ← jth consensus in ConsVctrs
        vj = consVctrs[j]
        if jaccard(vi, vj) == 1:
            st[i] += 1
            del st[j]
            del consVctrs[j]
            i -= 1
            
        j -= 1
        
    i -= 1
# 27 Find the consensus the most similar to the ensemble
# 28 L ← length(ConsVctrs)
L = len(consVctrs)
# 29 TSim ← Vector of ‘0’s of lengthL
tSim = [0] * L
# 30 for i = 1 to L do
for i in range(0, L):
    # 31 Ci ← ith consensus in ConsVctrs
    ci = consVctrs[i]
    # here tempArray will be ci converted from dict 
    #          {'maxDT=5|0': {1, 2, 3}, 'maxDT=5|1': {4, 5} ...
    # to array [0, 0, 0, 1, 1,...]
    tempArray = [-1] * len(baseClusterings.index)
    for indx, key in enumerate(ci):
        for ii in list(ci[key]):
            tempArray[baseClusterings.index.tolist().index(ii)] = indx
    # now tempArray is converted and equivalent to ci in the pseudo code
    # 32 for j = 1 to MaxDT do
    for j in range(maxDT):
        # 33 Cj ← jth clustering in BaseClusterings
        cj = baseClusterings.iloc[:,j]
        # 34 TSim[i] ← TSim[i] + Jaccard(Ci,Cj)
        tSim[i] += jaccard_score(tempArray, cj, average='macro')
    # 36 Sim[i] ← TSim[i] / MaxDT
    tSim[i] /= maxDT

for i in range(0, len(tSim)):
    selectedConsensus = consVctrs[i]
    # 38 RecommCons ← which.max(TSim)
    # we add 'selected' to the corresponding consensus
    isSelected = 'selected!' if i == tSim.index(max(tSim)) else ''
    clustAlgo_labels = pd.Series(name='consensus', index=baseClusterings.index, dtype='int')
    for ii, key in enumerate(selectedConsensus):
        for indx in selectedConsensus[key]:
            clustAlgo_labels[indx] = ii
            
#     score = customClusteringScore(encoders, clustAlgo_labels, testY)
#     print('stability=%i similarity=%f customScore=%f k=%i %s' % \
#           (st[i], tSim[i], score, clustAlgo_labels.nunique(), isSelected))
    print('stability=%i similarity=%f k=%i repartition=%s %s' % \
          (st[i], tSim[i], clustAlgo_labels.nunique(), str(consVctrs[i]), isSelected))

stability=1 similarity=0.163333 k=4 repartition={'maxDT=5|0': {1, 2, 3}, 'maxDT=5|1': {4, 5}, 'maxDT=5|2': {6, 7}, 'maxDT=5|3': {8, 9}} 
stability=2 similarity=0.217778 k=3 repartition={'maxDT=3|0': {1, 2, 3}, 'maxDT=3|1': {8, 9}, 'maxDT=3|2': {4, 5, 6, 7}} 
stability=1 similarity=0.462222 k=2 repartition={'maxDT=2|0': {1, 2, 3}, 'maxDT=2|1': {4, 5, 6, 7, 8, 9}} selected!
stability=1 similarity=0.162963 k=1 repartition={'maxDT=1|0': {1, 2, 3, 4, 5, 6, 7, 8, 9}} 


In [100]:
# trying to improve simirarity by rearrranging the partitions
# for example = for clustering [0,0,0,1,1,2,2] trying also:
# [1,1,1,0,0,2,2] , [2,2,2,1,1,0,0] , [0,0,0,2,2,1,1], etc...

L = len(consVctrs)
tSim = [0] * L
for i in range(0, L):
    ci = consVctrs[i]
    tempSimilaritysVec = []
    permutations = []
    for perm in itertools.permutations(ci):
        tempSimilarity = 0
        tempArray = [-1] * len(baseClusterings.index)
        indx = 0
        for key in perm:
            for ii in list(ci[key]):
                tempArray[baseClusterings.index.tolist().index(ii)] = indx
            indx += 1
        for j in baseClusterings.columns:
            cj = baseClusterings[j]
            tempSimilarity += jaccard_score(cj, tempArray, average='micro')
        tempSimilaritysVec.append(tempSimilarity/maxDT)
        permutations.append(perm)
    tSim[i] = max(tempSimilaritysVec)
    print('selected permutation =', permutations[tempSimilaritysVec.index(tSim[i])])

for i in range(0, len(tSim)):
    selectedConsensus = consVctrs[i]
    isSelected = 'selected!' if i == tSim.index(max(tSim)) else ''
    clustAlgo_labels = pd.Series(name='consensus', index=baseClusterings.index, dtype='int')
    for ii, key in enumerate(selectedConsensus):
        for indx in selectedConsensus[key]:
            clustAlgo_labels[indx] = ii
            
#     score = customClusteringScore(encoders, clustAlgo_labels, testY)
#     print('stability=%i similarity=%f customScore=%f k=%i %s' % \
#           (st[i], tSim[i], score, clustAlgo_labels.nunique(), isSelected))
    print('stability=%i similarity=%f k=%i repartition=%s %s' % \
          (st[i], tSim[i], clustAlgo_labels.nunique(), str(consVctrs[i]), isSelected))

selected permutation = ('maxDT=5|0', 'maxDT=5|2', 'maxDT=5|3', 'maxDT=5|1')
selected permutation = ('maxDT=3|2', 'maxDT=3|0', 'maxDT=3|1')
selected permutation = ('maxDT=2|0', 'maxDT=2|1')
selected permutation = ('maxDT=1|0',)
stability=1 similarity=0.331119 k=4 repartition={'maxDT=5|0': {1, 2, 3}, 'maxDT=5|1': {4, 5}, 'maxDT=5|2': {6, 7}, 'maxDT=5|3': {8, 9}} 
stability=2 similarity=0.457143 k=3 repartition={'maxDT=3|0': {1, 2, 3}, 'maxDT=3|1': {8, 9}, 'maxDT=3|2': {4, 5, 6, 7}} 
stability=1 similarity=0.476923 k=2 repartition={'maxDT=2|0': {1, 2, 3}, 'maxDT=2|1': {4, 5, 6, 7, 8, 9}} selected!
stability=1 similarity=0.271209 k=1 repartition={'maxDT=1|0': {1, 2, 3, 4, 5, 6, 7, 8, 9}} 
